In [1]:
import os
os.chdir("../")
%pwd

'h:\\Personal Projects\\Carbon-Footprint-estimation'

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("artifacts/data_ingestion/Carbon.csv")
data.head()

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,CarbonEmission
0,2,1,3,0,2,0,1,230,3,210,2,4,7,26,1,2,2238
1,3,1,1,3,1,1,1,114,1,9,3,3,9,38,5,2,1892
2,2,0,2,2,3,2,0,138,0,2472,0,1,14,47,6,0,2595
3,2,0,2,1,3,1,2,157,1,74,1,3,20,5,7,0,1074
4,3,1,1,0,2,2,1,266,2,8457,2,1,3,5,6,1,4743


In [4]:
data.dtypes

Body Type                        int64
Sex                              int64
Diet                             int64
How Often Shower                 int64
Heating Energy Source            int64
Transport                        int64
Social Activity                  int64
Monthly Grocery Bill             int64
Frequency of Traveling by Air    int64
Vehicle Monthly Distance Km      int64
Waste Bag Size                   int64
Waste Bag Weekly Count           int64
How Long TV PC Daily Hour        int64
How Many New Clothes Monthly     int64
How Long Internet Daily Hour     int64
Energy efficiency                int64
CarbonEmission                   int64
dtype: object

In [5]:
data.columns

Index(['Body Type', 'Sex', 'Diet', 'How Often Shower', 'Heating Energy Source',
       'Transport', 'Social Activity', 'Monthly Grocery Bill',
       'Frequency of Traveling by Air', 'Vehicle Monthly Distance Km',
       'Waste Bag Size', 'Waste Bag Weekly Count', 'How Long TV PC Daily Hour',
       'How Many New Clothes Monthly', 'How Long Internet Daily Hour',
       'Energy efficiency', 'CarbonEmission'],
      dtype='object')

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [7]:
from CarbonFootprint.constants import *
from CarbonFootprint.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = {}
        schema.update(self.schema["COLUMNS"])
        schema.update(self.schema["TARGET_COLUMN"])

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [9]:
import os
from CarbonFootprint import logger

In [10]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [11]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-05-13 11:03:20,164: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-13 11:03:20,167: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-13 11:03:20,168: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-13 11:03:20,168: INFO: common: created directory at: artifacts]
[2025-05-13 11:03:20,168: INFO: common: created directory at: artifacts/data_validation]
